In [1]:
import sys
import os
myDir = os.getcwd()
parentDir = os.path.abspath(os.path.join(myDir, os.pardir))
sys.path.append(parentDir)

from pathlib import Path
path = Path(parentDir)
a=str(path.parent.absolute())

sys.path.append(a)

In [2]:
from MetaX.utils.taxaFuncAnalyzer import TaxaFuncAnalyzer
from MetaX.utils.taxaFuncPloter.basic_plot import BasicPlot


In [3]:
df_path = './example_data/SW_TaxaFunc.tsv'
meta_path = './example_data/SW_Meta.tsv'
sw = TaxaFuncAnalyzer(df_path, meta_path)

original df shape: (3999, 229)
after remove all zero row: (3999, 229)


In [4]:
sw.set_func('Description')
sw.set_multi_tables(level='s')

sw.set_group('Person')

batch_list is not set, Batch effect removal did not perform.
transform_method is not set, data transform did not perform.
normalize_method is not set, data normalization did not perform.
Original data shape: (3999, 229)
Starting to set Function table...
Function number: 830
Starting to set Taxa table...
Taxa number: 340
Taxa-Function number: 1272
group is set to Person
 {'V4', 'V5', 'V1', 'V2', 'V3'}


In [14]:
from pydeseq2.ds import DeseqStats
from pydeseq2.dds import DeseqDataSet
import pandas as pd


def get_stats_deseq2(self, df, group_list: list):


    def replace_if_two_index(df):
        if isinstance(df.index, pd.MultiIndex):
            df = df.copy()
            df.reset_index(inplace=True)
            # DO NOT USE f-string here, it will cause error
            df['Taxa-Func'] = df.iloc[:,
                                        0].astype(str) + ' [' + df.iloc[:, 1].astype(str) + ']'
            df.set_index('Taxa-Func', inplace=True)
            df = df.drop(df.columns[:2], axis=1)
        else:
            df = df.copy()
        return df

    sample_list = []
    for i in group_list:
        sample = self.get_sample_list_in_a_group(i)
        sample_list += sample

    # Create intensity matrix
    df = df[sample_list]

    df = replace_if_two_index(df)
    

    counts_df = df.T
    # if the max value > 10000, divide by 100
    if counts_df.max().max() > 10000:
        counts_df = counts_df/100
    
    counts_df = counts_df.astype(int)
    display(counts_df)
    counts_df = counts_df.reindex(sample_list)
    display(counts_df)

    # Create meta data
    meta_df = self.meta_df.copy()
    meta_df = meta_df[meta_df['Sample'].isin(sample_list)]

    meta_df.set_index('Sample', inplace=True)
    meta_df = meta_df.reindex(sample_list)
    meta_df = meta_df.replace('_', '-', regex=True)

    dds = DeseqDataSet(
        counts=counts_df,
        clinical=meta_df,
        design_factors=self.meta_name,
        refit_cooks=True)

    dds.deseq2()
    stat_res = DeseqStats(dds)
    stat_res.summary()

    res = stat_res.results_df
    res_merged = pd.merge(res, df, left_index=True, right_index=True)

    return res_merged


df = get_stats_deseq2(sw, sw.taxa_func_df, ['V1', 'V2'])
df2 = get_stats_deseq2(sw, sw.taxa_func_df, ['V2', 'V1'])

Taxa-Func,d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium adolescentis [Belongs to the transketolase family],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium breve [D-xylulose 5-phosphate/D-fructose 6-phosphate phosphoketolase],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Binds to Cpn60 in the presence of Mg-ATP and suppresses the ATPase activity of the latter],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Binds to the 23S rRNA],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [DivIVA protein],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Essential cell division protein that forms a contractile ring structure (Z ring) at the future cell division site. The regulation of the ring assembly controls the timing and the location of cell division. One of the functions of the FtsZ ring is to recruit other cell division proteins to the septum to produce a new cell wall between the dividing cells. Binds GTP and shows GTPase activity],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Located at the back of the 30S subunit body where it stabilizes the conformation of the head with respect to the body],"d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Located at the top of the head of the 30S subunit, it contacts several helices of the 16S rRNA. In the 70S ribosome it contacts the 23S rRNA (bridge B1a) and protein L5 of the 50S subunit (bridge B1b), connecting the 2 subunits]",d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Participates in both transcription termination and antitermination],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Pyruvate formate lyase-like],...,d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Kluyvera|s__Kluyvera sp902363335 [Belongs to the membrane fusion protein (MFP) (TC 8.A.1) family],"d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Kluyvera|s__Kluyvera sp902363335 [Catalyzes the aldol condensation of dihydroxyacetone phosphate (DHAP or glycerone-phosphate) with glyceraldehyde 3- phosphate (G3P) to form fructose 1,6-bisphosphate (FBP) in gluconeogenesis and the reverse reaction in glycolysis]","d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Kluyvera|s__Kluyvera sp902363335 [first, biotin carboxylase catalyzes the carboxylation of the carrier protein and then the transcarboxylase transfers the carboxyl group to form malonyl-CoA]",d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Leclercia|s__Leclercia adecarboxylata_C [Forms part of the ribosomal stalk which helps the ribosome interact with GTP-bound translation factors. Is thus essential for accurate translation],d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Morganella|s__Morganella morganii [PTS HPr component phosphorylation site],"d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Morganella|s__Morganella morganii_A [Enolase, N-terminal domain]","d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__E

Taxa-Func,d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium adolescentis [Belongs to the transketolase family],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium breve [D-xylulose 5-phosphate/D-fructose 6-phosphate phosphoketolase],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Binds to Cpn60 in the presence of Mg-ATP and suppresses the ATPase activity of the latter],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Binds to the 23S rRNA],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [DivIVA protein],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Essential cell division protein that forms a contractile ring structure (Z ring) at the future cell division site. The regulation of the ring assembly controls the timing and the location of cell division. One of the functions of the FtsZ ring is to recruit other cell division proteins to the septum to produce a new cell wall between the dividing cells. Binds GTP and shows GTPase activity],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Located at the back of the 30S subunit body where it stabilizes the conformation of the head with respect to the body],"d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Located at the top of the head of the 30S subunit, it contacts several helices of the 16S rRNA. In the 70S ribosome it contacts the 23S rRNA (bridge B1a) and protein L5 of the 50S subunit (bridge B1b), connecting the 2 subunits]",d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Participates in both transcription termination and antitermination],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Pyruvate formate lyase-like],...,d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Kluyvera|s__Kluyvera sp902363335 [Belongs to the membrane fusion protein (MFP) (TC 8.A.1) family],"d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Kluyvera|s__Kluyvera sp902363335 [Catalyzes the aldol condensation of dihydroxyacetone phosphate (DHAP or glycerone-phosphate) with glyceraldehyde 3- phosphate (G3P) to form fructose 1,6-bisphosphate (FBP) in gluconeogenesis and the reverse reaction in glycolysis]","d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Kluyvera|s__Kluyvera sp902363335 [first, biotin carboxylase catalyzes the carboxylation of the carrier protein and then the transcarboxylase transfers the carboxyl group to form malonyl-CoA]",d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Leclercia|s__Leclercia adecarboxylata_C [Forms part of the ribosomal stalk which helps the ribosome interact with GTP-bound translation factors. Is thus essential for accurate translation],d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Morganella|s__Morganella morganii [PTS HPr component phosphorylation site],"d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Morganella|s__Morganella morganii_A [Enolase, N-terminal domain]","d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__E

Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.25 seconds.

Fitting dispersion trend curve...
... done in 0.40 seconds.

Fitting MAP dispersions...
... done in 0.37 seconds.

Fitting LFCs...
... done in 0.68 seconds.

Refitting 377 outliers.

Fitting dispersions...
... done in 0.10 seconds.

Fitting MAP dispersions...
... done in 0.10 seconds.

Fitting LFCs...
... done in 0.18 seconds.

Running Wald tests...
... done in 0.22 seconds.

Log2 fold change & Wald test p-value: Person V2 vs V1


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
Taxa-Func,,,,,,
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium adolescentis [Belongs to the transketolase family],888.193157,-17.343528,0.000000,0.000000,NaN,NaN
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium breve [D-xylulose 5-phosphate/D-fructose 6-phosphate phosphoketolase],798.485666,22.454256,13.867334,1.619219,1.054001e-01,1.829402e-01
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Binds to Cpn60 in the presence of Mg-ATP and suppresses the ATPase activity of the latter],47.821044,17.472294,12.368464,1.412649,1.577591e-01,2.545288e-01
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Binds to the 23S rRNA],6163.576031,3.119813,0.904769,3.448187,5.643639e-04,1.677018e-03
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [DivIVA protein],1767.823580,4.828015,0.837924,5.761875,8.318440e-09,4.851562e-08
...,...,...,...,...,...,...
"d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Morganella|s__Morganella morganii_A [Enolase, N-terminal domain]",685.389808,-28.271212,125.433431,-0.225388,8.216773e-01,8.909411e-01
"d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Serratia|s__Serratia ureilytica [Binds directly to 23S rRNA. The L1 stalk is quite mobile in the ribosome, and is involved in E site tRNA release]",1058.775042,-0.703427,2.885395,-0.243789,8.073945e-01,8.836171e-01
d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Succinivibrionaceae|g__Anaerobiospirillum|s__Anaerobiospirillum sp900543125 [Belongs to the universal ribosomal protein uS2 family],2852.278269,-17.448559,0.000000,0.000000,NaN,NaN


Taxa-Func,d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium adolescentis [Belongs to the transketolase family],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium breve [D-xylulose 5-phosphate/D-fructose 6-phosphate phosphoketolase],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Binds to Cpn60 in the presence of Mg-ATP and suppresses the ATPase activity of the latter],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Binds to the 23S rRNA],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [DivIVA protein],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Essential cell division protein that forms a contractile ring structure (Z ring) at the future cell division site. The regulation of the ring assembly controls the timing and the location of cell division. One of the functions of the FtsZ ring is to recruit other cell division proteins to the septum to produce a new cell wall between the dividing cells. Binds GTP and shows GTPase activity],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Located at the back of the 30S subunit body where it stabilizes the conformation of the head with respect to the body],"d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Located at the top of the head of the 30S subunit, it contacts several helices of the 16S rRNA. In the 70S ribosome it contacts the 23S rRNA (bridge B1a) and protein L5 of the 50S subunit (bridge B1b), connecting the 2 subunits]",d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Participates in both transcription termination and antitermination],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Pyruvate formate lyase-like],...,d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Kluyvera|s__Kluyvera sp902363335 [Belongs to the membrane fusion protein (MFP) (TC 8.A.1) family],"d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Kluyvera|s__Kluyvera sp902363335 [Catalyzes the aldol condensation of dihydroxyacetone phosphate (DHAP or glycerone-phosphate) with glyceraldehyde 3- phosphate (G3P) to form fructose 1,6-bisphosphate (FBP) in gluconeogenesis and the reverse reaction in glycolysis]","d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Kluyvera|s__Kluyvera sp902363335 [first, biotin carboxylase catalyzes the carboxylation of the carrier protein and then the transcarboxylase transfers the carboxyl group to form malonyl-CoA]",d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Leclercia|s__Leclercia adecarboxylata_C [Forms part of the ribosomal stalk which helps the ribosome interact with GTP-bound translation factors. Is thus essential for accurate translation],d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Morganella|s__Morganella morganii [PTS HPr component phosphorylation site],"d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Morganella|s__Morganella morganii_A [Enolase, N-terminal domain]","d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__E

Taxa-Func,d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium adolescentis [Belongs to the transketolase family],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium breve [D-xylulose 5-phosphate/D-fructose 6-phosphate phosphoketolase],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Binds to Cpn60 in the presence of Mg-ATP and suppresses the ATPase activity of the latter],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Binds to the 23S rRNA],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [DivIVA protein],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Essential cell division protein that forms a contractile ring structure (Z ring) at the future cell division site. The regulation of the ring assembly controls the timing and the location of cell division. One of the functions of the FtsZ ring is to recruit other cell division proteins to the septum to produce a new cell wall between the dividing cells. Binds GTP and shows GTPase activity],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Located at the back of the 30S subunit body where it stabilizes the conformation of the head with respect to the body],"d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Located at the top of the head of the 30S subunit, it contacts several helices of the 16S rRNA. In the 70S ribosome it contacts the 23S rRNA (bridge B1a) and protein L5 of the 50S subunit (bridge B1b), connecting the 2 subunits]",d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Participates in both transcription termination and antitermination],d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Pyruvate formate lyase-like],...,d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Kluyvera|s__Kluyvera sp902363335 [Belongs to the membrane fusion protein (MFP) (TC 8.A.1) family],"d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Kluyvera|s__Kluyvera sp902363335 [Catalyzes the aldol condensation of dihydroxyacetone phosphate (DHAP or glycerone-phosphate) with glyceraldehyde 3- phosphate (G3P) to form fructose 1,6-bisphosphate (FBP) in gluconeogenesis and the reverse reaction in glycolysis]","d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Kluyvera|s__Kluyvera sp902363335 [first, biotin carboxylase catalyzes the carboxylation of the carrier protein and then the transcarboxylase transfers the carboxyl group to form malonyl-CoA]",d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Leclercia|s__Leclercia adecarboxylata_C [Forms part of the ribosomal stalk which helps the ribosome interact with GTP-bound translation factors. Is thus essential for accurate translation],d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Morganella|s__Morganella morganii [PTS HPr component phosphorylation site],"d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Morganella|s__Morganella morganii_A [Enolase, N-terminal domain]","d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__E

Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.27 seconds.

Fitting dispersion trend curve...
... done in 0.35 seconds.

Fitting MAP dispersions...
... done in 0.34 seconds.

Fitting LFCs...
... done in 0.68 seconds.

Refitting 378 outliers.

Fitting dispersions...
... done in 0.10 seconds.

Fitting MAP dispersions...
... done in 0.10 seconds.

Fitting LFCs...
... done in 0.17 seconds.

Running Wald tests...
... done in 0.21 seconds.

Log2 fold change & Wald test p-value: Person V2 vs V1


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
Taxa-Func,,,,,,
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium adolescentis [Belongs to the transketolase family],888.193157,-12.197218,0.000000,0.000000,NaN,NaN
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium breve [D-xylulose 5-phosphate/D-fructose 6-phosphate phosphoketolase],798.485666,22.447262,13.837951,1.622152,1.047708e-01,1.852726e-01
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Binds to Cpn60 in the presence of Mg-ATP and suppresses the ATPase activity of the latter],47.821044,8.119045,2.911718,2.788403,5.296853e-03,1.329878e-02
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Binds to the 23S rRNA],6163.576031,3.119813,0.904787,3.448118,5.645071e-04,1.682769e-03
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [DivIVA protein],1767.823580,4.827890,0.837990,5.761277,8.348013e-09,4.992856e-08
...,...,...,...,...,...,...
"d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Morganella|s__Morganella morganii_A [Enolase, N-terminal domain]",685.389808,-28.271149,125.430606,-0.225393,8.216738e-01,8.929996e-01
"d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Serratia|s__Serratia ureilytica [Binds directly to 23S rRNA. The L1 stalk is quite mobile in the ribosome, and is involved in E site tRNA release]",1058.775042,-0.703427,2.885395,-0.243789,8.073945e-01,8.856816e-01
d__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Succinivibrionaceae|g__Anaerobiospirillum|s__Anaerobiospirillum sp900543125 [Belongs to the universal ribosomal protein uS2 family],2852.278269,-17.448559,0.000000,0.000000,NaN,NaN


In [15]:
display(df.head())
display(df2.head())

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,V1_01,V1_02,V1_03,V1_04,...,V2_29,V2_30,V2_31,V2_32,V2_33,V2_34,V2_35,V2_36,V2_37,V2_38
Taxa-Func,,,,,,,,,,,,,,,,,,,,,
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium adolescentis [Belongs to the transketolase family],888.193157,-17.343528,0.000000,0.000000,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium breve [D-xylulose 5-phosphate/D-fructose 6-phosphate phosphoketolase],798.485666,22.454256,13.867334,1.619219,1.054001e-01,1.829402e-01,0,0,0,0,...,0,452426,0,0,0,0,0,0,0,770676
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Binds to Cpn60 in the presence of Mg-ATP and suppresses the ATPase activity of the latter],47.821044,17.472294,12.368464,1.412649,1.577591e-01,2.545288e-01,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Binds to the 23S rRNA],6163.576031,3.119813,0.904769,3.448187,5.643639e-04,1.677018e-03,0,0,0,0,...,1700393,594603,1101556,468945,288604,1341208,1223687,1076967,915104,523734
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [DivIVA protein],1767.823580,4.828015,0.837924,5.761875,8.318440e-09,4.851562e-08,0,0,0,0,...,428624,226585,339502,235090,117053,318309,303443,308629,204507,149135


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,V2_01,V2_02,V2_03,V2_04,...,V1_29,V1_30,V1_31,V1_32,V1_33,V1_34,V1_35,V1_36,V1_37,V1_38
Taxa-Func,,,,,,,,,,,,,,,,,,,,,
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium adolescentis [Belongs to the transketolase family],888.193157,-12.197218,0.000000,0.000000,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium breve [D-xylulose 5-phosphate/D-fructose 6-phosphate phosphoketolase],798.485666,22.447262,13.837951,1.622152,1.047708e-01,1.852726e-01,0,0,730536,0,...,0,0,0,0,0,0,0,0,0,0
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Binds to Cpn60 in the presence of Mg-ATP and suppresses the ATPase activity of the latter],47.821044,8.119045,2.911718,2.788403,5.296853e-03,1.329878e-02,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [Binds to the 23S rRNA],6163.576031,3.119813,0.904787,3.448118,5.645071e-04,1.682769e-03,1208617,1255710,434118,945466,...,0,0,0,0,0,319441,0,0,0,0
d__Bacteria|p__Actinobacteriota|c__Actinomycetia|o__Actinomycetales|f__Bifidobacteriaceae|g__Bifidobacterium|s__Bifidobacterium infantis [DivIVA protein],1767.823580,4.827890,0.837990,5.761277,8.348013e-09,4.992856e-08,397368,319875,117728,429073,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from pydeseq2.ds import DeseqStats
from pydeseq2.dds import DeseqDataSet
import pandas as pd


def get_stats_deseq2(self, df, group_list: list):


    def replace_if_two_index(df):
        if isinstance(df.index, pd.MultiIndex):
            df = df.copy()
            df.reset_index(inplace=True)
            # DO NOT USE f-string here, it will cause error
            df['Taxa-Func'] = df.iloc[:,
                                        0].astype(str) + ' [' + df.iloc[:, 1].astype(str) + ']'
            df.set_index('Taxa-Func', inplace=True)
            df = df.drop(df.columns[:2], axis=1)
        else:
            df = df.copy()
        return df

    sample_list = []
    for i in group_list:
        sample = self.get_sample_list_in_a_group(i)
        sample_list += sample

    # Create intensity matrix
    df = df[sample_list]

    df = replace_if_two_index(df)
    

    counts_df = df.T
    # if the max value > 10000, divide by 100
    if counts_df.max().max() > 10000:
        counts_df = counts_df/100
    
    counts_df = counts_df.astype(int)
    display(counts_df.index)
    display(sample_list)
    counts_df = counts_df.sort_values(by=sample_list)

    display(counts_df)

    # Create meta data
    meta_df = self.meta_df.copy()
    meta_df = meta_df[meta_df['Sample'].isin(sample_list)]

    meta_df.set_index('Sample', inplace=True)
    meta_df = meta_df.sort_values(by=sample_list)
    meta_df = meta_df.replace('_', '-', regex=True)

    dds = DeseqDataSet(
        counts=counts_df,
        clinical=meta_df,
        design_factors=self.meta_name,
        refit_cooks=True)

    dds.deseq2()
    stat_res = DeseqStats(dds)
    stat_res.summary()

    res = stat_res.results_df
    res_merged = pd.merge(res, df, left_index=True, right_index=True)

    return res_merged


df = get_stats_deseq2(sw, sw.taxa_func_df, ['V1', 'V2'])
df2 = get_stats_deseq2(sw, sw.taxa_func_df, ['V2', 'V1'])

Index(['V1_01', 'V1_02', 'V1_03', 'V1_04', 'V1_05', 'V1_06', 'V1_07', 'V1_08',
       'V1_09', 'V1_10', 'V1_11', 'V1_12', 'V1_13', 'V1_14', 'V1_15', 'V1_16',
       'V1_17', 'V1_18', 'V1_19', 'V1_20', 'V1_21', 'V1_22', 'V1_23', 'V1_24',
       'V1_25', 'V1_26', 'V1_27', 'V1_28', 'V1_29', 'V1_30', 'V1_31', 'V1_32',
       'V1_33', 'V1_34', 'V1_35', 'V1_36', 'V1_37', 'V1_38', 'V2_01', 'V2_02',
       'V2_03', 'V2_04', 'V2_05', 'V2_06', 'V2_07', 'V2_08', 'V2_09', 'V2_10',
       'V2_11', 'V2_12', 'V2_13', 'V2_14', 'V2_15', 'V2_16', 'V2_17', 'V2_18',
       'V2_19', 'V2_20', 'V2_21', 'V2_22', 'V2_23', 'V2_24', 'V2_25', 'V2_26',
       'V2_27', 'V2_28', 'V2_29', 'V2_30', 'V2_31', 'V2_32', 'V2_33', 'V2_34',
       'V2_35', 'V2_36', 'V2_37', 'V2_38'],
      dtype='object')

['V1_01',
 'V1_02',
 'V1_03',
 'V1_04',
 'V1_05',
 'V1_06',
 'V1_07',
 'V1_08',
 'V1_09',
 'V1_10',
 'V1_11',
 'V1_12',
 'V1_13',
 'V1_14',
 'V1_15',
 'V1_16',
 'V1_17',
 'V1_18',
 'V1_19',
 'V1_20',
 'V1_21',
 'V1_22',
 'V1_23',
 'V1_24',
 'V1_25',
 'V1_26',
 'V1_27',
 'V1_28',
 'V1_29',
 'V1_30',
 'V1_31',
 'V1_32',
 'V1_33',
 'V1_34',
 'V1_35',
 'V1_36',
 'V1_37',
 'V1_38',
 'V2_01',
 'V2_02',
 'V2_03',
 'V2_04',
 'V2_05',
 'V2_06',
 'V2_07',
 'V2_08',
 'V2_09',
 'V2_10',
 'V2_11',
 'V2_12',
 'V2_13',
 'V2_14',
 'V2_15',
 'V2_16',
 'V2_17',
 'V2_18',
 'V2_19',
 'V2_20',
 'V2_21',
 'V2_22',
 'V2_23',
 'V2_24',
 'V2_25',
 'V2_26',
 'V2_27',
 'V2_28',
 'V2_29',
 'V2_30',
 'V2_31',
 'V2_32',
 'V2_33',
 'V2_34',
 'V2_35',
 'V2_36',
 'V2_37',
 'V2_38']

KeyError: 'V1_01'